In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from scipy.stats import describe
#from keras.utils import np_utils
#%matplotlib inline
import tensorflow as tf
import sys
from keras.backend.tensorflow_backend import set_session
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Concatenate, concatenate, Input, Reshape, LSTM, TimeDistributed
from keras.layers import LSTM, Dense, Conv1D, Input, Dropout, AvgPool1D, Reshape, Concatenate
from keras.layers.embeddings import Embedding
#import seaborn as sns
import warnings
from keras import models, layers
 # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")
print("python {}".format(sys.version))
print("keras version {}".format(keras.__version__))
print("tensorflow version {}".format(tf.__version__))

In [ ]:
# Loading the data
df_train  = pd.read_csv('../input/demand-forecasting-kernels-only/train.csv', parse_dates=['date'])
df_test = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv', parse_dates=['date'])
sample_sub = pd.read_csv('../input/demand-forecasting-kernels-only/sample_submission.csv')
print('Train shape:{}, Test shape:{}'.format(df_train.shape, df_test.shape))

In [ ]:
# data processing
import os, sys, time, math, json, random
#import tensorflow as tf
import matplotlib.pyplot as plt


def set_seed(sd=123): #define seed for consistent output
    from numpy.random import seed
    from tensorflow import set_random_seed
    import random as rn
    ## numpy random seed
    seed(sd)
    ## core python's random number
    rn.seed(sd)
    ## tensor flow's random number
    set_random_seed(sd)
#set_seed(sd=123)
import sys
from itertools import product, starmap

def storeitems():
    return product(range(1,51), range(1,11))


def storeitems_column_names():
    return list(starmap(lambda i,s: f'item_{i}_store_{s}_sales', storeitems()))


def sales_by_storeitem(df):
    ret = pd.DataFrame(index=df.index.unique())
    for i, s in storeitems():
        #print(i,s)
        #sys.exit()

        ret[f'item_{i}_store_{s}_sales'] = df[(df['item'] == i) & (df['store'] == s)]['sales'].values
        #print( ret)
    return ret

def shift_series(series, days):
    return series.transform(lambda x: x.shift(days))


def shift_series_in_df(df, series_names=[], days_delta=90):
    ret = pd.DataFrame(index=df.index.copy())
    str_sgn = 'future' if np.sign(days_delta) < 0 else 'past'
    for sn in series_names:
        ret[f'{sn}_{str_sgn}_{np.abs(days_delta)}'] = shift_series(df[sn], days_delta)
    return ret


def stack_shifted_sales(df, days_deltas=[1, 90, 360]):
    names = storeitems_column_names()
    dfs = [df.copy()]
    #print(dfs)
    #sys.exit()
    for delta in days_deltas:
        shifted = shift_series_in_df(df, series_names=names, days_delta=delta)
        #print(shifted.shape)
        #sys.exit()
        dfs.append(shifted)
    return pd.concat(dfs, axis=1, sort=False, copy=False)

def generate_data(df_train, df_test ):
    df_train.index = pd.to_datetime(df_train['date'])
    df_train.drop('date', axis=1, inplace=True)
    df_train = sales_by_storeitem(df_train)

    df_test.index = pd.to_datetime(df_test['date'])
    df_test.drop('date', axis=1, inplace=True)
    df_test['sales'] = np.zeros(df_test.shape[0])
    df_test = sales_by_storeitem(df_test)

    col_names = list(zip(df_test.columns, df_train.columns))  #('item_1_store_1_sales', 'item_1_store_1_sales')
    for cn in col_names:
        assert cn[0] == cn[1]

    df_test['is_test'] = np.repeat(True, df_test.shape[0])
    df_train['is_test'] = np.repeat(False, df_train.shape[0])
    df_total = pd.concat([df_train, df_test])
    weekday_df = pd.get_dummies(df_total.index.weekday, prefix='weekday')
    weekday_df.index = df_total.index
    month_df = pd.get_dummies(df_total.index.month, prefix='month')
    month_df.index =  df_total.index
    df_total = pd.concat([weekday_df, month_df, df_total], axis=1)
    assert df_total.isna().any().any() == False
    df_total = stack_shifted_sales(df_total, days_deltas=[1])
    df_total.dropna(inplace=True)
    sales_cols = [col for col in df_total.columns if '_sales' in col and '_sales_' not in col]
    stacked_sales_cols = [col for col in df_total.columns if '_sales_' in col]
    other_cols = [col for col in df_total.columns if col not in set(sales_cols) and col not in set(stacked_sales_cols)]

    sales_cols = sorted(sales_cols)
    stacked_sales_cols = sorted(stacked_sales_cols)

    new_cols = other_cols + stacked_sales_cols + sales_cols
    df_total = df_total.reindex(columns=new_cols)
    assert df_total.isna().any().any() == False
    scaler = MinMaxScaler(feature_range=(0,1))
    cols_to_scale = [col for col in df_total.columns if 'weekday' not in col and 'month' not in col and "is_test" not in col]
    scaled_cols = scaler.fit_transform(df_total[cols_to_scale])
    df_total[cols_to_scale] = scaled_cols
    df_train = df_total[df_total['is_test'] == False].drop('is_test', axis=1)
    df_test = df_total[df_total['is_test'] == True].drop('is_test', axis=1)

    X_cols_stacked = [col for col in df_train.columns if '_past_' in col]
    X_cols_caldata = [col for col in df_train.columns if 'weekday_' in col or 'month_' in col or 'year' in col]
    X_cols = X_cols_stacked + X_cols_caldata

    X = df_train[X_cols]
    y_cols = [col for col in df_train.columns if col not in X_cols]
    y = df_train[y_cols]
    X_test = df_test[X_cols]
    return X, y, X_test, scaler, df_test, cols_to_scale, y_cols

In [ ]:
def define_model2(time_steps,
                    hidden_neurons = 50,
                    batch_size=None,
                    feature_dim =20,
                    stateful=False):
    output_neurons = 1

    inputs = Input( batch_shape=(batch_size,time_steps, feature_dim) )  #Input(shape=(X_train.shape[1], X_train.shape[2]))

    top_lstm  = LSTM( hidden_neurons,activation='sigmoid',kernel_initializer='normal', dropout=0.25, return_sequences=True, stateful=stateful, name="lstm")(inputs)
    top_dense = TimeDistributed( Dense(50, activation='relu'))(top_lstm)
    top_dropout = TimeDistributed(Dropout(50))(top_dense)
    # bottom pipeline
    bottom_dense = TimeDistributed(Dense(50))(inputs)
    bottom_conv1 = Conv1D(
                            50,
                            kernel_size=1,
                            input_shape=(batch_size, time_steps, feature_dim)#input_shape=(X_train.shape[1], X_train.shape[2])
                        )(bottom_dense)
    bottom_conv2 = Conv1D(
                    100,
                    kernel_size=50,
                    padding='same',
                    activation='relu'
                )(bottom_conv1)
    bottom_conv3 = Conv1D(
                    50,
                    kernel_size=10,
                    padding='same',
                    activation='relu'
                )(bottom_conv2)
    bottom_pooling = AvgPool1D(
                    pool_size=60,
                    padding='same'
                )(bottom_conv3)
    bottom_reshape = TimeDistributed(Reshape(
                    target_shape=[50]
                ))(bottom_conv3)
    # concat output from both pipelines
    final_concat = Concatenate()([top_dropout, bottom_reshape])

    target_model = TimeDistributed(Dense(output_neurons,name="dense"))(final_concat)
    model = models.Model(inputs=inputs, outputs = target_model )

    model.compile(loss="mean_squared_error",sample_weight_mode="temporal",optimizer="rmsprop")

    return model,target_model




def define_model(time_steps,
                    hidden_neurons = 50,
                    batch_size=None,
                    feature_dim =20,
                    stateful=False):
    output_neurons = 1

    inputs = Input(batch_shape=(batch_size,time_steps, feature_dim))  #Input(shape=(X_train.shape[1], X_train.shape[2]))

    lstm  = LSTM( hidden_neurons,activation='sigmoid',kernel_initializer='normal', dropout=0.25, return_sequences=True, stateful=stateful, name="lstm")(inputs)

    target_model = TimeDistributed(Dense(output_neurons,name="dense"))(lstm)
    model = models.Model(inputs=inputs, outputs = target_model )

    model.compile(loss="mean_squared_error",sample_weight_mode="temporal",optimizer="rmsprop")

    return (model,(inputs,lstm,target_model))
#https://fairyonice.github.io/Stateful-LSTM-model-training-in-Keras.html
class statefulModel(object):
    def __init__(self,model,print_val_every = 500):
        '''
        model must be stateful keras model object
        batch_input_shape must be specified
        '''
        bis = model.layers[0].get_config()["batch_input_shape"]
        print("batch_input_shape={}".format(bis))
        self.batch_size = bis[0]
        self.ts         = bis[1]
        self.Nfeat      = bis[2]
        self.model      = model
        self.print_val_every = print_val_every

    def train1epoch(self, X, y, epoch=None):
        '''
        devide the training set of time series into batches.
        '''
        print( "  Training..")
        batch_index = np.arange(X.shape[0])
        ## shuffle to create batch containing different time series
        np.random.shuffle(batch_index)
        count = 1

        #print("batch_index",batch_index)
        for ibatch in range(self.batch_size,X.shape[0]+1, self.batch_size):

            print( "    Batch {:02d}".format(count))
            #print( "ibatch",ibatch )
            pick = batch_index[(ibatch-self.batch_size):ibatch]
            #print("pick", pick  , len(pick))
            if len(pick) < self.batch_size:
                continue

            X_batch = X[pick]
            y_batch = y[pick]
            #print("count", count)
            #print(X1_batch.shape)
            #ibatch = 50

            self.fit_across_time(X_batch,y_batch,epoch,ibatch)
            count += 1
            #print("count", count)

    def fit_across_time(self, X, y, epoch=None, ibatch=None):
        '''
        training for the given set of time series
        It always starts at the time point 0 so we need to reset states to zero.
        '''
        self.model.reset_states()

        for itime in range(self.ts,X.shape[1]+1,self.ts):
            ## extract sub time series
            print("itime",itime)

            Xtime = X[:,itime-self.ts:itime,:]
            ytime = y[:,itime-self.ts:itime,:]

            #print(X1.shape, X1time.shape ,ytime.shape )

            val = self.model.fit(Xtime,ytime,
                        nb_epoch=1,
                        ## no shuffling across rows (i.e. time series)
                        shuffle=False,
                        ## use all the samples in one epoch
                        batch_size=Xtime.shape[0],
                        verbose= False)
            if itime % self.print_val_every == 0:
                print( "      {start:4d}:{end:4d} loss={val:.3f}".format(
                start=itime-self.ts, end=itime, val=val.history["loss"][0]))
                sys.stdout.flush()
                ## uncomment below if you do not want to save weights for every epoch every batch and every time
        if epoch is not None:
            #path = "../output" ../output/

            self.model.save_weights(
                "weights_epoch{:03d}_batch{:01d}.hdf5".format(epoch+1, ibatch))

    def fit(self, X, y, epochs=300):

        past_val_loss = np.Inf
        history = []
        for iepoch in range(epochs):
            self.model.reset_states()
            print( "__________________________________")
            print( "Epoch {}".format(iepoch+1))

            self.train1epoch( X, y, iepoch)
        return history

def stateful_prediction( mm,X_test, ntarget=1):
        #expecting..
        bis = mm.layers[0].get_config()["batch_input_shape"]
        batch_size, ts, nfeat = bis
        assert(X_test.shape[0] % batch_size == 0)
        assert(X_test.shape[1] % ts == 0)

        y_pred = np.zeros((X_test.shape[0],X_test.shape[1],ntarget))
        #y_pred[:] = np.NaN

        for ipat in range(0,X_test.shape[0],batch_size):
            #mm.reset_states()
            for itime in range(0, X_test.shape[1], ts):
                X_testi = X_test[ipat:(ipat+batch_size),itime:(itime+ts),:]
                y_pred[ipat:(ipat+batch_size),itime:(itime+ts),:] = mm.predict(X_testi,batch_size = batch_size)
                y_pred_temp =  np.squeeze(y_pred[ipat:(ipat+batch_size),itime:(itime+ts),:])
                y_pred_temp= y_pred_temp.reshape((y_pred_temp.shape[0],1))
                #print( X_testi[:,:,0].shape, y_pred_temp.shape)

                #sys.exit()

                X_testi[:,:,0] = y_pred_temp#y_pred[ipat:(ipat+batch_size),itime:(itime+ts),:][:,:]
        return y_pred
def unscale(y_arr, scaler, template_df, toint=False):
    """
    Unscale array y_arr of model predictions, based on a scaler fitted 
    to template_df.
    """
    tmp = template_df.copy()
    tmp[y_cols] = pd.DataFrame(y_arr, index=tmp.index)
    tmp[cols_to_scale] = scaler.inverse_transform(tmp[cols_to_scale])
    if toint:
        return tmp[y_cols].astype(int)
    return tmp

In [ ]:
X_train, y_train, X_test, scaler, df_test, cols_to_scale, y_cols = generate_data(df_train, df_test )

X_trans = X_train.transpose()
y_train = y_train.transpose()
y = y_train.values.reshape((500,1825,1))
X_trans.info()

N= 500
T= 1825
D= 20
x_train = np.zeros((N,T,D))

import sys
for n in range(N+19):
    for t in range(T):
        if n<500:
            x_train[n,t,0] = X_trans.iloc[n][t]
        else: 
            for d in range(1,D):
                x_train[:,t,d] = X_trans.iloc[N+d-1][t]

hunits = 50
time_steps = 365
epochs = 10
print_val_every = 365
batch_size = 25
model_stateful, _ = define_model(time_steps,
                    hidden_neurons = hunits,
                    batch_size = batch_size,
                    stateful = True
                     )
#model_stateless.summary()

smodel = statefulModel(model=model_stateful,print_val_every = print_val_every)

#X = [X1_train,X2_train,X3_train,X4_train,X5_train,X6_train]
#X_val = [X1_val,X2_val,X3_val,X4_val,X5_val,X6_val]

start = time.time()
history_stateful = smodel.fit(x_train, y, epochs)

end = time.time()
print("Time Took {:3.2f} min".format((end - start)/60))
sys.exit()

In [ ]:
X_test = X_test.transpose()
N= 500
T= 90
D= 20
x_test_pros = np.zeros((N,T,D))

import sys
for n in range(N+19):
    for t in range(T):
        if n<500:
            x_test_pros[n,t,0] = X_test.iloc[n][t]
        else: 
            for d in range(1,D):
                x_test_pros[:,t,d] = X_test.iloc[N+d-1][t]

In [ ]:
def define_model3(time_steps,
                    hidden_neurons = 50,
                    batch_size=None,
                    feature_dim =20,
                    stateful=False):
    output_neurons = 1

    inputs = Input( batch_shape=(batch_size,time_steps, feature_dim) )  #Input(shape=(X_train.shape[1], X_train.shape[2]))

    top_lstm  = LSTM( hidden_neurons,activation='sigmoid',kernel_initializer='normal', dropout=0.25, return_sequences=True, stateful=stateful, name="lstm")(inputs)
    top_dense = TimeDistributed( Dense(hidden_neurons, activation='relu'))(top_lstm)
    top_dropout = TimeDistributed(Dropout(0.5))(top_dense)
    # bottom pipeline
    bottom_dense = TimeDistributed(Dense(hidden_neurons))(inputs)
    bottom_conv1 = Conv1D(
                            hidden_neurons,
                            kernel_size=1,
                            input_shape=(batch_size, time_steps, feature_dim)#input_shape=(X_train.shape[1], X_train.shape[2])
                        )(bottom_dense)
    bottom_conv2 = Conv1D(
                    50,
                    kernel_size=50,
                    padding='same',
                    activation='relu'
                )(bottom_conv1)
    bottom_conv3 = Conv1D(
                    25,
                    kernel_size=10,
                    padding='same',
                    activation='relu'
                )(bottom_conv2)
    bottom_pooling = AvgPool1D(
                    pool_size=5,
                    padding='same'
                )(bottom_conv3)
    bottom_reshape = TimeDistributed(Reshape(
                    target_shape=[hidden_neurons]
                ))(bottom_conv3)
    # concat output from both pipelines
    final_concat = Concatenate()([top_dropout, bottom_reshape])

    target_model = TimeDistributed(Dense(output_neurons,name="dense"))(final_concat)
    model = models.Model(inputs=inputs, outputs = target_model )

    model.compile(loss="mean_squared_error",sample_weight_mode="temporal",optimizer="rmsprop")

    return model,target_model
#https://fairyonice.github.io/Stateful-LSTM-model-training-in-Keras.html


In [ ]:
hunits = 20
time_steps = 1
epochs = 20
print_val_every = 365
batch_size = 25
model_pred1, _ = define_model(time_steps,
                    hidden_neurons = hunits,
                    batch_size = batch_size,
                    stateful = True
                     )


model_pred1.load_weights("../input/weight-ts6/weights_epoch011_batch500.hdf5")

In [ ]:
y_pred_stateful1 = stateful_prediction(mm = model_pred1,X_test = x_test_pros)
y_pred_stateful122= np.squeeze(y_pred_stateful1 )

In [ ]:
template_df = df_test
template_df['is_test'] = np.repeat(True, template_df.shape[0])
basic_pred = unscale(y_pred_stateful122.transpose(), scaler, template_df, toint=True)

In [ ]:
import itertools
def sales_by_storeitem(basic_pred):
    a=[]
    
    for  i, s in product(range(1,51), range(1,11) ):
        #print("store, item",s,i)
        a.append( basic_pred[f'item_{i}_store_{s}_sales'].values  )
        #print(len(set(basic_pred.index.values) ))
        #print((basic_pred.index[basic_pred[f'item_{i}_store_{s}_sales']] ))
        #ids = (basic_pred.index[basic_pred[f'item_{i}_store_{s}_sales']].tolist() )  #item_9_store_7_sales
        #print(basic_pred[f'item_{i}_store_{s}_sales'])
        #print(basic_pred[f'item_{i}_store_{s}_sales'].values)
        #print(len(set(ids)))
    a= list(itertools.chain.from_iterable(a))

    return a

submission3= sales_by_storeitem( basic_pred)

In [ ]:
for idx, row in sample_sub.iterrows():
    #print(idx,row['store'],row['item'])
    #sys.exit()
    row["sales"] =submission3[idx] #y_pred[idx] #np.round(y_pred[idx]).astype(int)
sample_sub.to_csv("submission.csv", index=False)
print( " submission successful " )